In [1]:
# system packages
import sys
import time
import warnings
import os

# non-geo numeric packages
import numpy as np
import math
from itertools import product, combinations
import pandas as pd

# network and OSM packages
import networkx as nx
import osmnx as ox
city_geo = ox.geocoder.geocode_to_gdf

# Earth engine packages
import ee
import geemap

# General geo-packages
import libpysal
import rasterio
import geopandas as gpd
import shapely
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString, Polygon, MultiPolygon

In [12]:
city_geo('Kumamoto City')

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((130.02576 32.31688, 130.02594 32.316...",32.32,32.32,130.03,130.03,278956318,way,924413802,32.32,130.03,"Amakusa City Hall, Kawaura Tomitsu Branch, Sak...",amenity,townhall,0.20


In [2]:
# Authenticate and Initialize Google Earth Engine
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AbUR2VPNtosClEEvAYVUM6aovF9ZgWy9FW-Y61YmsBL7yno94EEnbtHAYP4

Successfully saved authorization token.


In [41]:
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(ls_in)]
cities_adj = cities_adj.sort_values('City').reset_index()

In [42]:
cities_adj

,index,OSM_area,City,E2SFCA,Gravity,Unnamed: 4,Unnamed: 5,G_adj,E_adj,Unnamed: 8
0,0,Abidjan,Abidjan,X,X,X,1.00,NaN,NaN,NaN
1,1,Addis Ababa,Addis Ababa,X,X,X,NaN,NaN,NaN,NaN
2,5,Baghdad,Baghdad,X,X,X,1.00,NaN,NaN,NaN
3,6,Bangalore Urban,Bangalore,X,X,X,NaN,NaN,NaN,NaN
4,8,Beograd,Belgrade,X,X,X,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
70,103,Utsunomiya,Utsunomiya,X,X,X,NaN,NaN,NaN,NaN
71,106,Washington DC,Washington DC,X,X,X,NaN,NaN,NaN,NaN
72,107,"Fushan district Yantai, Zhifu district Yantai,...",Yantai,X,X,X,0.00,NaN,NaN,NaN
73,108,Yaounde,Yaounde,X,X,X,NaN,NaN,NaN,NaN


In [13]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Kumamoto'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'])
print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext = '_Kumamoto')

E2SCFA_score['score summary']

['Japan']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\JPN_Kumamoto_2020.tif
get road networks from OSM
Kumamoto done 2.94 mns
 
get urban greenspaces from OSM
Kumamoto done
 
100m resolution grids extraction
Kumamoto 2.21 mns

get fake UGS entry points
Kumamoto 0.0 % done 0.0  mns
Kumamoto 26.8 % done 0.48  mns
Kumamoto 53.6 % done 0.88  mns
Kumamoto 80.4 % done 1.28  mns
Kumamoto 100 % done 1.57  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
34.18 % 0.38 mns
68.35 % 0.81 mns
100 % finding combinations done
Kumamoto 341618 suitible combinations

obtain local graphs
Kumamoto
0.0 % done 0.78 mns
34.18 % done 0.82 mns
68.35 % done 1.39 mns
100 % done 45.61 mns

Kumamoto
0.0 % 0.14 mns
9.68 % 0.52 mns
19.36 % 0.89 mns
29.04 % 1.27 mns
38.72 % 1.65 mns
48.4 % 2.03 mns
58.08 % 2.39 mns
67.76 % 2.77 mns
77.44 % 3.15 mns
87.12 % 3.5 mns
96.8 % 3.77 mns
100 % done 4.03 mns

300 Kumamoto
600 Kumamoto
1000 Kumamoto
CPU times: total: 10min 33s
Wall time

City,Kumamoto
0,"843,427.00"
Sc-access 300,61.42
M-dist 300,9.44
M-area 300,"29,409.06"
M-supply 300,45.37
Sc-norm 300,29.63
Sc-access 600,60.36
M-dist 600,59.47
M-area 600,"47,604.54"
M-supply 600,53.54


In [13]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Manila'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'],
                            time_sleep = 30)

print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext = '_Manila')

E2SCFA_score['score summary']

['Philippines']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\PHL_Manila_2020.tif
get road networks from OSM
Manila done 2.48 mns
 
get urban greenspaces from OSM
Manila done
 
100m resolution grids extraction
Manila 3.06 mns

get fake UGS entry points
Manila 0.0 % done 0.01  mns
Manila 8.8 % done 1.14  mns
Manila 17.6 % done 2.12  mns
Manila 26.4 % done 3.14  mns
Manila 35.2 % done 4.31  mns
Manila 44.0 % done 5.44  mns
Manila 52.8 % done 6.6  mns
Manila 61.6 % done 7.79  mns
Manila 70.4 % done 8.88  mns
Manila 79.2 % done 9.99  mns
Manila 88.0 % done 11.1  mns
Manila 96.8 % done 12.35  mns
Manila 100 % done 12.81  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
13.09 % 0.44 mns
26.18 % 0.92 mns
39.27 % 1.44 mns
52.36 % 1.92 mns
65.45 % 2.41 mns
78.53 % 2.92 mns
91.62 % 3.45 mns
100 % finding combinations done
Manila 364181 suitible combinations

obtain local graphs
Manila
0.0 % done 5.96 mns
13.09 % done 6.22 mns
26.18 % done 6.86 mns
39.27 % 

City,Manila
0,"13,219,247.00"
Sc-access 300,15.57
M-dist 300,17.93
M-area 300,"600,837.24"
M-supply 300,12.38
Sc-norm 300,0.86
Sc-access 600,13.29
M-dist 600,118.35
M-area 600,"807,341.35"
M-supply 600,11.36


In [31]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Tallinn'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds,
                                time_sleep = 10)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'])
print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext = '_Tallinn')

E2SCFA_score['score summary']

['Estonia']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\EST_Tallinn_2020.tif
get road networks from OSM
Tallinn done 1.72 mns
 
get urban greenspaces from OSM
Tallinn done
 
100m resolution grids extraction
Tallinn 3.69 mns

get fake UGS entry points
Tallinn 0.0 % done 0.01  mns
Tallinn 24.0 % done 0.97  mns
Tallinn 48.1 % done 1.65  mns
Tallinn 72.1 % done 2.28  mns
Tallinn 96.2 % done 2.91  mns
Tallinn 100 % done 3.16  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
5.84 % 0.25 mns
11.68 % 0.55 mns
17.52 % 0.89 mns
23.36 % 1.28 mns
29.21 % 1.71 mns
35.05 % 2.19 mns
40.89 % 2.69 mns
46.73 % 3.24 mns
52.57 % 3.84 mns
58.41 % 4.49 mns
64.25 % 5.17 mns
70.09 % 5.95 mns
75.93 % 6.79 mns
81.78 % 7.6 mns
87.62 % 8.45 mns
93.46 % 9.33 mns
99.3 % 10.24 mns
100 % finding combinations done
Tallinn 4436294 suitible combinations

obtain local graphs
Tallinn
0.0 % done 5.9 mns
5.84 % done 6.21 mns
11.68 % done 6.77 mns
17.52 % done 7.35 mns
23.36 % done 8

City,Tallinn
0,"416,024.00"
Sc-access 300,597.76
M-dist 300,52.04
M-area 300,"1,200,232.45"
M-supply 300,246.87
Sc-norm 300,321.10
Sc-access 600,530.10
M-dist 600,194.96
M-area 600,"1,639,596.00"
M-supply 600,248.46


In [11]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Kumamoto'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'])
print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext = '_Kumamoto')

E2SCFA_score['score summary']

['Japan']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\JPN_Kumamoto_2020.tif
get road networks from OSM
Kumamoto done 0.91 mns
 
get urban greenspaces from OSM
Kumamoto done
 
100m resolution grids extraction
Kumamoto 3.16 mns

get fake UGS entry points
Kumamoto 0.0 % done 0.0  mns
Kumamoto 70.9 % done 0.21  mns
Kumamoto 100 % done 0.3  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
88.03 % 0.13 mns
100 % finding combinations done
Kumamoto 70922 suitible combinations

obtain local graphs
Kumamoto
0.0 % done 0.14 mns
88.03 % done 0.18 mns
100 % done 0.68 mns

Kumamoto
0.0 % 0.11 mns
56.07 % 0.42 mns
100 % done 0.67 mns

300 Kumamoto
600 Kumamoto
1000 Kumamoto
CPU times: total: 4min 59s
Wall time: 6min 5s


City,Kumamoto
0,"125,461.00"
Sc-access 300,41.62
M-dist 300,6.24
M-area 300,"3,225.59"
M-supply 300,52.58
Sc-norm 300,10.80
Sc-access 600,41.42
M-dist 600,46.31
M-area 600,"7,866.09"
M-supply 600,48.14


In [12]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Perth'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds,
                                time_sleep = 10)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'])
print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext = '_Perth')

E2SCFA_score['score summary']

['Australia']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\AUS_Perth_2020.tif
get road networks from OSM
Perth done 5.66 mns
 
get urban greenspaces from OSM
Perth done
 
100m resolution grids extraction
Perth 9.92 mns

get fake UGS entry points
Perth 0.0 % done 0.01  mns
Perth 5.0 % done 1.62  mns
Perth 9.9 % done 2.73  mns
Perth 14.9 % done 3.8  mns
Perth 19.8 % done 4.85  mns
Perth 24.8 % done 5.97  mns
Perth 29.7 % done 7.02  mns
Perth 34.7 % done 8.12  mns
Perth 39.7 % done 9.16  mns
Perth 44.6 % done 10.24  mns
Perth 49.6 % done 11.23  mns
Perth 54.5 % done 12.37  mns
Perth 59.5 % done 13.44  mns
Perth 64.5 % done 14.47  mns
Perth 69.4 % done 15.51  mns
Perth 74.4 % done 16.51  mns
Perth 79.3 % done 17.53  mns
Perth 84.3 % done 18.53  mns
Perth 89.2 % done 19.55  mns
Perth 94.2 % done 20.64  mns
Perth 99.2 % done 21.69  mns
Perth 100 % done 22.29  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
2.62 % 0.39 mns
5.24 % 0.78 mns
7.86 % 1.19 

In [30]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Lubumbashi'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'])
print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext = '_Lubumbashi')

E2SCFA_score['score summary']

['Democratic Republic of the Congo']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\COD_Lubumbashi_2020.tif
get road networks from OSM
Lubumbashi done 0.76 mns
 
get urban greenspaces from OSM
Lubumbashi done
 
100m resolution grids extraction
Lubumbashi 0.84 mns

get fake UGS entry points
Lubumbashi 0.0 % done 0.0  mns
Lubumbashi 100 % done 0.04  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
100 % finding combinations done
Lubumbashi 23550 suitible combinations

obtain local graphs
Lubumbashi
0.0 % done 0.02 mns
100 % done 0.03 mns

Lubumbashi
0.0 % 0.06 mns
100 % done 0.31 mns

300 Lubumbashi
600 Lubumbashi
1000 Lubumbashi
CPU times: total: 2min 11s
Wall time: 2min 30s


City,Lubumbashi
0,"4,009,290.00"
Sc-access 300,0.02
M-dist 300,0.39
M-area 300,51.69
M-supply 300,0.02
Sc-norm 300,0.00
Sc-access 600,0.02
M-dist 600,3.57
M-area 600,149.39
M-supply 600,0.03


In [40]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Curitiba'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'])
print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext ='_Curitiba')

E2SCFA_score['score summary']

['Brazil']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\BRA_Curitiba_2020.tif
get road networks from OSM
Curitiba done 1.6 mns
 
get urban greenspaces from OSM
Curitiba done
 
100m resolution grids extraction
Curitiba 5.19 mns

get fake UGS entry points
Curitiba 0.0 % done 0.04  mns
Curitiba 10.8 % done 0.82  mns
Curitiba 21.7 % done 1.55  mns
Curitiba 32.5 % done 2.25  mns
Curitiba 43.3 % done 2.9  mns
Curitiba 54.2 % done 3.56  mns
Curitiba 65.0 % done 4.27  mns
Curitiba 75.8 % done 4.98  mns
Curitiba 86.7 % done 5.7  mns
Curitiba 97.5 % done 6.42  mns
Curitiba 100 % done 6.64  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
9.45 % 0.36 mns
18.9 % 0.73 mns
28.35 % 1.11 mns
37.8 % 1.5 mns
47.25 % 1.9 mns
56.7 % 2.33 mns
66.15 % 2.79 mns
75.6 % 3.26 mns
85.05 % 3.77 mns
94.5 % 4.26 mns
100 % finding combinations done
Curitiba 1010783 suitible combinations

obtain local graphs
Curitiba
0.0 % done 4.15 mns
9.45 % done 4.25 mns
18.9 % done 4.34 mn

City,Curitiba
0,"2,054,010.00"
Sc-access 300,66.46
M-dist 300,39.39
M-area 300,"91,448.58"
M-supply 300,45.56
Sc-norm 300,33.35
Sc-access 600,55.54
M-dist 600,197.52
M-area 600,"125,504.12"
M-supply 600,40.23


In [32]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Yaounde'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'])
print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext ='_Yaounde')

E2SCFA_score['score summary']

['Cameroon']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\CMR_Yaounde_2020.tif
get road networks from OSM
Yaounde done 0.71 mns
 
get urban greenspaces from OSM
Yaounde done
 
100m resolution grids extraction
Yaounde 1.92 mns

get fake UGS entry points
Yaounde 0.0 % done 0.01  mns
Yaounde 44.6 % done 0.48  mns
Yaounde 89.3 % done 0.84  mns
Yaounde 100 % done 0.97  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
36.31 % 0.22 mns
72.62 % 0.47 mns
100 % finding combinations done
Yaounde 446602 suitible combinations

obtain local graphs
Yaounde
0.0 % done 0.43 mns
36.31 % done 0.48 mns
72.62 % done 0.55 mns
100 % done 0.62 mns

Yaounde
0.0 % 0.12 mns
8.52 % 0.48 mns
17.03 % 0.85 mns
25.55 % 1.24 mns
34.06 % 1.6 mns
42.58 % 1.94 mns
51.09 % 2.29 mns
59.61 % 2.6 mns
68.12 % 2.93 mns
76.64 % 3.3 mns
85.15 % 3.68 mns
93.67 % 4.0 mns
100 % done 4.25 mns

300 Yaounde
600 Yaounde
1000 Yaounde
CPU times: total: 6min 42s
Wall time: 9min 45s


City,Yaounde
0,"3,265,872.00"
Sc-access 300,"3,670.07"
M-dist 300,12.61
M-area 300,"57,797,987.94"
M-supply 300,"3,641.46"
Sc-norm 300,"2,619.49"
Sc-access 600,"3,231.83"
M-dist 600,78.85
M-area 600,"61,225,156.21"
M-supply 600,"3,246.59"


In [31]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Guatemala City'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'])
print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext ='_Guatemala City')

E2SCFA_score['score summary']

['Guatemala']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\GTM_Guatemala City_2020.tif
get road networks from OSM
Guatemala City done 1.34 mns
 
get urban greenspaces from OSM
Guatemala City done
 
100m resolution grids extraction
Guatemala City 1.82 mns

get fake UGS entry points
Guatemala City 0.0 % done 0.0  mns
Guatemala City 11.5 % done 0.42  mns
Guatemala City 23.0 % done 0.87  mns
Guatemala City 34.5 % done 1.32  mns
Guatemala City 46.0 % done 1.77  mns
Guatemala City 57.5 % done 2.22  mns
Guatemala City 69.0 % done 2.69  mns
Guatemala City 80.6 % done 3.12  mns
Guatemala City 92.1 % done 3.55  mns
Guatemala City 100 % done 3.87  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
31.85 % 0.3 mns
63.69 % 0.59 mns
95.54 % 0.9 mns
100 % finding combinations done
Guatemala City 47007 suitible combinations

obtain local graphs
Guatemala City
0.0 % done 0.85 mns
31.85 % done 0.88 mns
63.69 % done 0.91 mns
95.54 % done 0.95 mns
100 % done 0.96 mns

City,Guatemala City
0,"2,373,141.00"
Sc-access 300,797.06
M-dist 300,4.69
M-area 300,"792,500.26"
M-supply 300,770.71
Sc-norm 300,70.73
Sc-access 600,778.97
M-dist 600,26.35
M-area 600,"901,640.23"
M-supply 600,788.54


In [36]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Pune'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'])
print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext ='_Pune')

E2SCFA_score['score summary']

['India']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\IND_Pune_2020.tif
get road networks from OSM
Pune done 1.3 mns
 
get urban greenspaces from OSM
Pune done
 
100m resolution grids extraction
Pune 0.55 mns

get fake UGS entry points
Pune 0.0 % done 0.0  mns
Pune 27.5 % done 0.6  mns
Pune 55.1 % done 1.18  mns
Pune 82.6 % done 1.71  mns
Pune 100 % done 2.06  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
51.87 % 0.23 mns
100 % finding combinations done
Pune 64441 suitible combinations

obtain local graphs
Pune
0.0 % done 0.7 mns
51.87 % done 0.75 mns
100 % done 0.8 mns

Pune
0.0 % 0.11 mns
46.9 % 0.45 mns
93.8 % 0.71 mns
100 % done 0.96 mns

300 Pune
600 Pune
1000 Pune
CPU times: total: 5min 31s
Wall time: 6min 23s


City,Pune
0,"5,081,607.00"
Sc-access 300,19.92
M-dist 300,10.54
M-area 300,"220,643.84"
M-supply 300,18.30
Sc-norm 300,0.67
Sc-access 600,17.86
M-dist 600,78.36
M-area 600,"283,859.52"
M-supply 600,17.23


In [37]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['Chengdu'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'])
print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext ='_Chengdu')

E2SCFA_score['score summary']

['China']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\CHN_Chengdu_2020.tif
get road networks from OSM
Chengdu done 1.29 mns
 
get urban greenspaces from OSM
Chengdu done
 
100m resolution grids extraction
Chengdu 2.19 mns

get fake UGS entry points
Chengdu 0.0 % done 0.0  mns
Chengdu 17.9 % done 0.32  mns
Chengdu 35.8 % done 0.66  mns
Chengdu 53.8 % done 0.99  mns
Chengdu 71.7 % done 1.3  mns
Chengdu 89.6 % done 1.6  mns
Chengdu 100 % done 1.78  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
32.18 % 0.34 mns
64.35 % 0.68 mns
96.53 % 1.04 mns
100 % finding combinations done
Chengdu 143214 suitible combinations

obtain local graphs
Chengdu
0.0 % done 0.53 mns
32.18 % done 0.67 mns
64.35 % done 0.82 mns
96.53 % done 0.97 mns
100 % done 0.99 mns

Chengdu
0.0 % 0.1 mns
42.8 % 0.45 mns
85.59 % 0.72 mns
100 % done 0.97 mns

300 Chengdu
600 Chengdu
1000 Chengdu
CPU times: total: 7min 43s
Wall time: 8min 42s


City,Chengdu
0,"6,218,911.00"
Sc-access 300,122.64
M-dist 300,7.76
M-area 300,"256,418.08"
M-supply 300,116.18
Sc-norm 300,41.07
Sc-access 600,122.61
M-dist 600,61.68
M-area 600,"353,290.13"
M-supply 600,125.59


In [38]:
%%time
# Thresholds and cities
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract cities list
iso = pd.read_excel('iso_countries.xlsx')
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(['San Antonio'])]
cities_adj = cities_adj.reset_index()

# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj,iso,'D:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

# 2. Information extraction

# Get road networks
road_networks = road_network(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')
# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

print(' ')
# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     road_networks['nodes'],
                                     UGS,
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print('')
# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           road_networks['graphs'],
                           cities_adj['City'],
                           population_grids,
                           thresholds,
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print('')
suitible_enh = suitible_enhanced(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], 
                                 cities_adj['City'], 
                                 thresholds)
print('')
subgraphs = obtaining_subgraphs(road_networks['graphs'],
                                population_grids,
                                UGS_entry,
                                road_networks['nodes'],
                                cities_adj['City'],
                                thresholds)
print('')
Dir_Routes = direct_routing (suitible_enh,
                             subgraphs['graphs'],
                             road_networks['edges'],
                             cities_adj['City'])
print('')
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Dir_Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA_adj/', 
                             grid_size = 100,
                             ext ='_San Antonio')

E2SCFA_score['score summary']

['United States']
Generating URL ...
Please wait ...
Data downloaded to D:\Dumps\GEE_city_grids\USA_San Antonio_2020.tif
get road networks from OSM
San Antonio done 3.73 mns
 
get urban greenspaces from OSM
San Antonio done
 
100m resolution grids extraction
San Antonio 10.41 mns

get fake UGS entry points
San Antonio 0.0 % done 0.11  mns
San Antonio 14.0 % done 1.56  mns
San Antonio 28.0 % done 3.01  mns
San Antonio 42.0 % done 4.43  mns
San Antonio 56.0 % done 5.78  mns
San Antonio 70.0 % done 7.26  mns
San Antonio 84.0 % done 8.57  mns
San Antonio 98.0 % done 9.88  mns
San Antonio 100 % done 10.13  mns

get (Euclidean) suitible combinations
0.0 % 0.0 mns
18.83 % 0.94 mns
37.65 % 1.89 mns
56.48 % 2.85 mns
75.3 % 3.8 mns
94.13 % 4.77 mns
100 % finding combinations done
San Antonio 209691 suitible combinations

obtain local graphs
San Antonio
0.0 % done 4.56 mns
18.83 % done 4.62 mns
37.65 % done 4.69 mns
56.48 % done 4.78 mns
75.3 % done 4.87 mns
94.13 % done 4.97 mns
100 % done 5.01 

City,San Antonio
0,"2,168,868.00"
Sc-access 300,845.87
M-dist 300,4.93
M-area 300,"214,639.84"
M-supply 300,921.52
Sc-norm 300,579.78
Sc-access 600,689.16
M-dist 600,39.07
M-area 600,"360,971.62"
M-supply 600,860.53


In [3]:
def gee_worldpop_extract (city_file, iso, save_path = None):
    
    cities = city_file
    iso['name'] = np.where(iso['name'] == 'Macedonia','North Macedonia',iso['name'])
    
    # Get included city areas
    OSM_incl = [cities[cities['City'] == city]['OSM_area'].tolist()[0].rsplit(', ') for city in cities['City'].tolist()]

    # Get the city geoms
    obj = [city_geo(city).dissolve()['geometry'].tolist()[0] for city in OSM_incl]
    
    # Get the city countries
    obj_displ = [city_geo(city).dissolve()['display_name'].tolist()[0].rsplit(', ')[-1]for city in OSM_incl]
    print(obj_displ)
    obj_displ = np.where(pd.Series(obj_displ).str.contains("Ivoire"),"CIte dIvoire",obj_displ)

    # Get the country's iso-code
    iso_list = [iso[iso['name'] == ob]['alpha3'].tolist()[0] for ob in obj_displ]

    # Based on the iso-code return the worldpop 2020
    ee_worldpop = [ee.ImageCollection("WorldPop/GP/100m/pop")\
        .filter(ee.Filter.date('2020'))\
        .filter(ee.Filter.inList('country', [io])).first() for io in iso_list]

    # Clip the countries with the city geoms.
    clipped = [ee_worldpop[i].clip(shapely.geometry.mapping(obj[i])) for i in range(0,len(obj))]

    # Create path if non-existent
    if save_path == None:
        path = ''
    else:
        path = save_path
        if not os.path.exists(path):
                    os.makedirs(path)

    # Export as TIFF file.
    # Stored in form path + USA_Los Angeles_2020.tif
    filenames = [path+iso_list[i]+'_'+cities['City'][i]+'_2020.tif' for i in range(len(obj))]
    [geemap.ee_export_image(clipped[i], filename = filenames[i]) for i in range(0,len(obj))]
    return(filenames)
    sys.stdout.flush()
    
    # Block 2 Road networks
def road_network (cities, thresholds, undirected = False):
    print('get road networks from OSM')
    start_time = time.time()
    graphs = list()
    road_nodes = list()
    road_edges = list()
    road_conn = list()

    for i in enumerate(cities['OSM_area']):
        # Get graph, road nodes and edges
        road_node = pd.DataFrame()
        roads = pd.DataFrame()
        
        # For each included OSM_area get the roads
        for district in i[1].rsplit(', '):
            graph = ox.graph_from_place(district, network_type = "all", buffer_dist = (np.max(thresholds)+1000))
            node, edge = ox.graph_to_gdfs(graph)
            road_node = pd.concat([road_node, node], axis = 0)
            roads = pd.concat([roads, edge], axis = 0)
        
        # Eliminate lists in the df which prevents drop of duplicate columns
        road_edge = pd.DataFrame([[c[0] if isinstance(c,list) else c for c in roads[col]]\
                              for col in roads]).transpose()
        road_edge.columns = roads.columns
        road_edge.index = roads.index
        road_edge = gpd.GeoDataFrame(road_edge, crs = 4326)
        
        # Return the unique nodes and edges of the (often) adjacent OSM_areas.
        road_node = road_node.drop_duplicates()
        road_edge = road_edge.drop_duplicates()
        
        # Road nodes format
        road_node = road_node.to_crs(4326)
        road_node['geometry_m'] = gpd.GeoSeries(road_node['geometry'], crs = 4326).to_crs(3043)
        road_node['osmid_var'] = road_node.index
        road_node = gpd.GeoDataFrame(road_node, geometry = 'geometry', crs = 4326)

        # format road edges
        road_edge['geometry_m'] = gpd.GeoSeries(road_edge['geometry'], crs = 4326).to_crs(3043)
        road_edge = road_edge.reset_index()
        road_edge.rename(columns={'u':'from', 'v':'to', 'key':'keys'}, inplace=True)
        road_edge['key'] = road_edge['from'].astype(str) + '-' + road_edge['to'].astype(str)
        
        if undirected == True:
            # Apply one-directional to both for walking
            both = road_edge[road_edge['oneway'] == False]
            one = road_edge[road_edge['oneway'] == True]
            rev = pd.DataFrame()
            rev[['from','to']] = one[['to','from']]
            rev = pd.concat([rev,one.iloc[:,2:]],axis = 1)
            edge_bidir = pd.concat([both, one, rev])
            edge_bidir = edge_bidir.reset_index()
            edge_bidir['oneway'] = False
        else:
            edge_bidir = road_edge

        # Exclude highways and ramps on edges    
        edge_filter = edge_bidir[(edge_bidir['highway'].str.contains('motorway') | 
              (edge_bidir['highway'].str.contains('trunk') & 
               edge_bidir['maxspeed'].astype(str).str.contains(
                   '40 mph|45 mph|50 mph|55 mph|60 mph|65|70|75|80|85|90|95|100|110|120|130|140'))) == False]
        road_edges.append(edge_filter)

        # Exclude isolated nodes
        fltrnodes = pd.Series(list(edge_filter['from']) + list(edge_filter['to'])).unique()
        newnodes = road_node[road_node['osmid_var'].isin(fltrnodes)]
        road_nodes.append(newnodes)

        # Get only necessary road connections columns for network performance
        road_con = edge_filter[['osmid','key','length','geometry']]
        road_con = road_con.set_index('key')

        road_conn.append(road_con)

        # formatting to graph again.
        newnodes = newnodes.loc[:, ~newnodes.columns.isin(['geometry_m', 'osmid_var'])]
        edge_filter = edge_filter.set_index(['from','to','keys'])
        edge_filter = edge_filter.loc[:, ~edge_filter.columns.isin(['geometry_m', 'key'])]

        graph2 = ox.graph_from_gdfs(newnodes, edge_filter)

        graphs.append(graph2)
        print(cities['City'][i[0]].rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')
    return({'graphs':graphs,'nodes':road_nodes,'edges':road_conn,'edges long':road_edges})
# Block 3 city greenspace
def urban_greenspace (cities, thresholds, one_UGS_buf = 25, min_UGS_size = 400):
    print('get urban greenspaces from OSM')
    parks_in_range = list()
    for i in enumerate(cities['OSM_area']):
        # Tags seen as Urban Greenspace (UGS) require the following:
        # 1. Tag represent an area
        # 2. The area is outdoor
        # 3. The area is (semi-)publically available
        # 4. The area is likely to contain trees, grass and/or greenery
        # 5. The area can reasonable be used for walking or recreational activities
        tags = {'landuse':['allotments','forest','greenfield','village_green'],\
                'leisure':['garden','fitness_station','nature_reserve','park','playground'],\
                'natural':'grassland'}
        gdf = ox.geometries_from_place(i[1].rsplit(', '),tags = tags,buffer_dist = np.max(thresholds))
        gdf = gdf[(gdf.geom_type == 'Polygon') | (gdf.geom_type == 'MultiPolygon')]
        greenspace = gdf.reset_index()    
        warnings.filterwarnings("ignore")

        green_buffer = gpd.GeoDataFrame(geometry = greenspace.to_crs(3043).buffer(one_UGS_buf).to_crs(4326))
        greenspace['geometry_w_buffer'] = green_buffer
        greenspace['geometry_w_buffer'] = gpd.GeoSeries(greenspace['geometry_w_buffer'], crs = 4326)
        greenspace['geom buffer diff'] = greenspace['geometry_w_buffer'].difference(greenspace['geometry'])

        # This function group components in itself that overlap (with the buffer set of 25 metres)
        # https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
        W = libpysal.weights.fuzzy_contiguity(greenspace['geometry_w_buffer'])
        greenspace['components'] = W.component_labels
        parks = greenspace.dissolve('components')

        # Exclude parks below 0.04 ha.
        parks = parks[parks.to_crs(3043).area > min_UGS_size]
        print(cities['City'][i[0]], 'done')
        parks = parks.reset_index()
        parks['geometry_m'] = parks['geometry'].to_crs(3043)
        parks['park_area'] = parks['geometry_m'].area
        parks_in_range.append(parks)
    return(parks_in_range)
# Block 4 population grids extraction
def city_grids_format(city_grids, cities_area, road_nodes, UGS, grid_size = 100):
    start_time = time.time()
    grids = []
    print(str(grid_size) + 'm resolution grids extraction')
    for i in range(len(city_grids)):
        
        # Open the raster file
        with rasterio.open(city_grids[i]) as src:
            band= src.read() # the population values
            aff = src.transform # the raster bounds and size (affine)
        
        # Get the rowwise arrays, get a 2D dataframe
        grid = pd.DataFrame()
        for b in enumerate(band[0]):
            grid = pd.concat([grid, pd.Series(b[1],name=b[0])],axis=1)
        grid= grid.unstack().reset_index()
        
        # Unstack df to columns
        grid.columns = ['row','col','value']
        grid['minx'] = aff[2]+aff[0]*grid['col']
        grid['miny'] = aff[5]+aff[4]*grid['row']
        grid['maxx'] = aff[2]+aff[0]*grid['col']+aff[0]
        grid['maxy'] = aff[5]+aff[4]*grid['row']+aff[4]
        
        # Create polygon from affine bounds and row/col indices
        grid['geometry'] = [Polygon([(grid.minx[i],grid.miny[i]),
                                   (grid.maxx[i],grid.miny[i]),
                                   (grid.maxx[i],grid.maxy[i]),
                                   (grid.minx[i],grid.maxy[i])])\
                          for i in range(len(grid))]
        
        # Set the df as geo-df
        grid = gpd.GeoDataFrame(grid, crs = 4326) 

        # Get dissolvement_key for dissolvement. 
        grid['row3'] = np.floor(grid['row']/(grid_size/100)).astype(int)
        grid['col3'] = np.floor(grid['col']/(grid_size/100)).astype(int)
        grid['dissolve_key'] = grid['row3'].astype(str) +'-'+ grid['col3'].astype(str)
        
        # Define a city's OSM area as Polygon.
        geo_ls = gpd.GeoSeries(city_geo(cities_area[i].split(', ')).dissolve().geometry)
        
        # Intersect grids with the city boundary Polygon.
        insec = grid.intersection(geo_ls.tolist()[0])
        
        # Exclude grids outside the specified city boundaries
        insec = insec[insec.area > 0]
        
        # Join in other information.
        insec = gpd.GeoDataFrame(geometry = insec, crs = 4326).join(grid.loc[:, grid.columns != 'geometry'])
        
        # Dissolve into block by block grids
        popgrid = insec[['dissolve_key','geometry','row3','col3']].dissolve('dissolve_key')
        
        # Get those grids populations and area. Only blocks with population and full blocks
        popgrid['population'] = round(insec.groupby('dissolve_key')['value'].sum()).astype(int)
        popgrid['area_m'] = round(gpd.GeoSeries(popgrid['geometry'], crs = 4326).to_crs(3043).area).astype(int)
        popgrid = popgrid[popgrid['population'] > 0]
        popgrid = popgrid[popgrid['area_m'] / popgrid['area_m'].max() > 0.95]

        # Get centroids and coords
        popgrid['centroid'] = popgrid['geometry'].centroid
        popgrid['centroid_m'] = gpd.GeoSeries(popgrid['centroid'], crs = 4326).to_crs(3043)
        popgrid['grid_lon'] = popgrid['centroid_m'].x
        popgrid['grid_lat'] = popgrid['centroid_m'].y
        popgrid = popgrid.reset_index()

        minx = popgrid.bounds['minx']
        maxx = popgrid.bounds['maxx']
        miny = popgrid.bounds['miny']
        maxy = popgrid.bounds['maxy']

        # Some geometries result in a multipolygon when dissolving (like i.e. 0.05 meters), coords error.
        # Therefore recreate the polygon.
        Poly = []
        for k in range(len(popgrid)):
            Poly.append(Polygon([(minx[k],maxy[k]),(maxx[k],maxy[k]),(maxx[k],miny[k]),(minx[k],miny[k])]))
        popgrid['geometry'] = Poly
        
        try:
            entry_index = [int(road_nodes[i]['geometry'].sindex.nearest(grid)[1])\
                                 for grid in popgrid['centroid']]
        except:
            entry_index = [int(road_nodes[i]['geometry'].sindex.nearest(grid)[1][0])\
                                 for grid in popgrid['centroid']]
            
        nearest_index = road_nodes[i].iloc[entry_index]
        popgrid['grid_osm'] = nearest_index.reset_index(drop = True)['osmid_var']
        popgrid['node_geom'] = nearest_index.reset_index(drop = True)['geometry']
        popgrid['node_geom_m'] = nearest_index.reset_index(drop = True)['geometry_m']
        popgrid['G-entry cost'] = popgrid['node_geom_m'].distance(popgrid['centroid_m'])
        
        UGS_all = UGS[i].dissolve().geometry[0]
        popgrid['in_out_UGS'] = popgrid.intersection(UGS_all).is_empty == False
        
        grids.append(popgrid)

        print(city_grids[i].rsplit('_')[3], round((time.time() - start_time)/60,2),'mns')
    return(grids)

In [4]:
# Block 5 park entry points
def UGS_fake_entry(UGS, road_nodes, graphs, cities, pop_grids,
                   thresholds, UGS_entry_buf = 25, walk_radius = 500, entry_point_merge = 0):
    print('get fake UGS entry points')
    start_time = time.time()
    ParkRoads = list()
    for j in range(len(cities)):
        ParkRoad = pd.DataFrame()
        mat = list()
        # For all
        for i in range(len(UGS[j])):
            dist = road_nodes[j]['geometry'].to_crs(3043).distance(UGS[j]['geometry'].to_crs(
                3043)[i])
            buf_nodes = road_nodes[j][(dist < UGS_entry_buf) & (dist > 0)]
            mat.append(list(np.repeat(i, len(buf_nodes))))
            ParkRoad = pd.concat([ParkRoad, buf_nodes])
            if i % 100 == 0: print(cities[j].rsplit(',')[0], round(i/len(UGS[j])*100,1),'% done', 
                                  round((time.time() - start_time) / 60,2),' mns')
        # Park no list conversion
        mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

        # Format
        ParkRoad['Park_No'] = mat_u
        ParkRoad = ParkRoad.reset_index()
        ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
        ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
        
        # Get the road nodes intersecting with the parks' buffer
        ParkRoad = pd.merge(ParkRoad, UGS[j][['geometry','park_area']], left_on = 'Park_No', right_index = True)

        # Get the walkable park size
        ParkRoad['park_size_walkable'] = ParkRoad['geometry_m'].buffer(walk_radius).to_crs(4326).intersection(ParkRoad['geometry_y'].to_crs(4326))
        ParkRoad['walk_area'] = ParkRoad['park_size_walkable'].to_crs(3043).area
        ParkRoad['park_area'] = ParkRoad['geometry_y'].to_crs(3043).area
        ParkRoad['share_walked'] = ParkRoad['walk_area'] / ParkRoad['park_area']
                
        # Merge fake UGS entry points if within X meters of each other for better system performance
        # Standard no merging
        ParkRoad = simplify_UGS_entry(ParkRoad, entry_point_merge = 0)
                
        ParkRoads.append(ParkRoad)

        print(cities[j].rsplit(',')[0],'100 % done', 
                                  round((time.time() - start_time) / 60,2),' mns')
        
    return(ParkRoads)
# Block 5.5 (not in use, buffer is 0, thus retains all the park entry points as is)
def simplify_UGS_entry(fake_UGS_entry, entry_point_merge = 0):
    # Get buffer of nodes close to each other.
    # Get the buffer
    ParkComb = fake_UGS_entry
    ParkComb['geometry_m_buffer'] = ParkComb['geometry_m'].buffer(entry_point_merge)

    # Get and merge components
    M = libpysal.weights.fuzzy_contiguity(ParkComb['geometry_m_buffer'])
    ParkComb['components'] = M.component_labels

    # Take centroid of merged components
    centr = gpd.GeoDataFrame(ParkComb, geometry = 'geometry_x', crs = 4326).dissolve('components')['geometry_x'].centroid
    centr = gpd.GeoDataFrame(centr)
    centr.columns = ['comp_centroid']

    # Get node closest to the centroid of all merged nodes, which accesses the road network.
    ParkComb = pd.merge(ParkComb, centr, left_on = 'components', right_index = True)
    ParkComb['centr_dist'] = ParkComb['geometry_x'].distance(ParkComb['comp_centroid'])
    ParkComb = ParkComb.iloc[ParkComb.groupby('components')['centr_dist'].idxmin()]
    return(ParkComb)

In [5]:
def suitible_enhanced (UGS_entry, pop_grids, road_nodes, cities, thresholds):
    start_time = time.time()
    suits_all = []
    for j in range(len(cities)):
        print('get (Euclidean) suitible combinations')
        print('0.0 %', round((time.time() - start_time) / 60,2),'mns')
        UGSe = UGS_entry[j]
        entry_geoms = UGSe.geometry_m
        pop = pop_grids[j]
        road_node = road_nodes[j]

        suits = pd.DataFrame()
        cols = ['osmid','Park_No','park_area']
        for i in range(len(entry_geoms)):
            suit_df = pop[pop.node_geom_m.distance(entry_geoms.iloc[i]) < np.max(thresholds)]
        
            suit_df['UGSe_osmid_m'] = entry_geoms.iloc[i]
            suit_df['Grid_No'] = suit_df.index
            suit_df = suit_df[['Grid_No','grid_osm','G-entry cost','in_out_UGS','node_geom_m','UGSe_osmid_m']].reset_index(drop = True)
            suit_df['Park_entry_No'] = UGSe.index[i]
            suits = pd.concat([suits,suit_df])
            if (i+1) % 500 == 0: print(round((i+1) / len(entry_geoms)*100,2),'%',
                                       round((time.time() - start_time) / 60,2),'mns')
            
        suits = pd.merge(suits, UGSe[cols], left_on = 'Park_entry_No',right_index = True, how = 'left')
        suits = suits.reset_index(drop = True)
        suits = suits.rename(columns = {'osmid':'Parkroad_osmid','park_area':'park_area_m2'})
        suits['gridpark_no'] = suits['Grid_No'].astype(str)+'-'+suits['Park_No'].astype(str)
        suits['graph_key'] = suits['grid_osm'].astype(str)+'-'+suits['Parkroad_osmid'].astype(str)
        suits_all.append(suits)
        print('100 % finding combinations done')
        print(cities[j],len(suits),'suitible combinations')
    return(suits_all)

In [6]:
def obtaining_subgraphs(graphs, pop_grids, UGS_entry, nodes, cities, thresholds, time_sleep = 30):
    print('obtain local graphs')
    start_time = time.time()
    subgraphs_all = []
    suits_all = []
    for j in range(len(cities)):
        print(cities[j])
        Graph = graphs[j]
        pop = pop_grids[j]
        UGSe = UGS_entry[j].sort_values('osmid')
        road_node = nodes[j]
        node_geoms = road_node.geometry_m
        entry_geoms = UGSe.geometry_m
        osmid = UGSe['osmid']

        dist = [node_geoms.distance(Point(i)) for i in entry_geoms]

        print('0.0 % done',round((time.time() - start_time) / 60,2),'mns')
        subgraphs = []
        UGSe_ids = []
        suits = pd.DataFrame()
        for i in range(len(entry_geoms)):      
            suit = road_node[['geometry_m']]
            suit['UGSe_osmid_m'] = entry_geoms.iloc[i]
            suit_df = dist[i]
            suit_in = suit_df[suit_df <= max(thresholds)]
            UGSe_ids.append(osmid.iloc[i])
            suit_in = pd.DataFrame(suit_in).join(node_geoms)
            suit_in['Parkroad_osmid'] = osmid.iloc[i]
            subgraphs.append(Graph.subgraph(suit_in.index))
            suits = pd.concat([suits, suit_in])

            if (i+1) % 500 == 0: 
                print(round((i+1) / len(entry_geoms)*100,2),'% done',
                                        round((time.time() - start_time) / 60,2),'mns')
                time.sleep(time_sleep)
        print('100 % done',round((time.time() - start_time) / 60,2),'mns')
        subgraphs_all.append(pd.Series(subgraphs, index = UGSe_ids))
        suits_all.append(suits)
    return({'graphs':subgraphs_all,'graph nodes':suits_all})

In [7]:
def distance_fast (Geo_1, Geo_2):
    return((abs(Geo_1.x - Geo_2.x)**2 + abs(Geo_1.y - Geo_2.y)**2).apply(math.sqrt))

In [8]:
def direct_routing (suitible_comb, graphs, edges, cities, chunk = 20000, time_sleep = 15):
    start_time = time.time()
    Routes = []
    Lines = []
    for j in enumerate(cities):
        print(j[1])
        
        suitible = suitible_comb[j[0]].sort_values('Parkroad_osmid').reset_index()
        grouped = suitible[suitible['in_out_UGS'] == False].groupby(['Parkroad_osmid'])['grid_osm'].apply(list)
        sets = grouped.apply(np.unique)

        Conn = edges[j[0]]
        SG = graphs[j[0]]
        SG = SG[sets.index]
        
        SGr = SG.reset_index()
        SG = SGr.iloc[pd.Series(SGr['Parkroad_osmid'].drop_duplicates()).index].set_index('Parkroad_osmid')[0]

        num = int(np.ceil(chunk / sets.apply(len).mean()))
        length = int(np.ceil(len(suitible['Parkroad_osmid'].unique())/num))

        Routes_df = pd.DataFrame()
        Lines_df = pd.DataFrame()
        for l in range(length):
            comb = suitible[suitible['Parkroad_osmid'].isin(sets.index[l*num:l*num+num])]
            sets2 = sets[l*num:l*num+num]

            parknode = list(comb['Parkroad_osmid'])
            gridnode = list(comb['grid_osm'])
            subgraph = SG[sets2.index]
            #sets2 = sets2[subgraph.index]

            ls = []
            ls2 = []
            ls3 = []
            lod = []
            lgk = []
            Routes
            for i in range(len(sets2)):
                path = nx.single_source_dijkstra(subgraph.iloc[i], sets2.index[i], weight = 'length')

                incl = np.isin(list(path[0].keys()),sets2.iloc[i])
                incl2 = np.isin(list(path[1].keys()),sets2.iloc[i])

                # route cost
                orig_c = list(np.repeat(sets2.index[i],sum(incl)))
                dest_c = list(np.array(list(path[0].keys()))[incl])
                cost = list(np.array(list(path[0].values()))[incl])

                ls = ls + orig_c
                ls2= ls2+ dest_c
                ls3= ls3+ cost

                # route steps
                orig_s = list(np.repeat(sets2.index[i],sum(incl2)))
                dest_s = list(np.array(list(path[1].keys()))[incl2])
                steps = list(np.array(list(path[1].values()),dtype=object)[incl2])

                fr = []
                to = []
                og = []
                de = []
                for j in enumerate(steps):
                    if len(j[1]) > 1:
                        fr.append(j[1][:-1])
                        to.append(j[1][1:])
                        og.append(list(np.repeat(orig_s[j[0]], len(j[1][:-1]))))
                        de.append(list(np.repeat(dest_s[j[0]], len(j[1][:-1]))))
                    else:
                        pass

                fr = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, fr) for i in b]
                to = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, to) for i in b]
                og = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, og) for i in b]
                de = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, de) for i in b]

                gk = [str(fr[k])+'-'+str(to[k]) for k in range(len(to))]
                gkr = [str(to[k])+'-'+str(fr[k]) for k in range(len(to))]
                od = [str(de[k])+'-'+str(og[k]) for k in range(len(og))]

                lgk.append(gk)
                lod.append(od)

            dist_df = pd.DataFrame({'UGSe_id':ls,'GrE_id':ls2,'route cost':ls3})
            dist_df['graph_key'] = dist_df['GrE_id'].astype(str)+'-'+dist_df['UGSe_id'].astype(str)

            routes = pd.merge(comb, dist_df, on = 'graph_key', how = 'left')
            routes['route cost'] = np.where(routes['in_out_UGS'],0,routes['route cost'])
            routes = routes[~routes['route cost'].isna()].reset_index(drop = True)

            routes['G-entry cost'] = np.where(routes['in_out_UGS'],0,routes['G-entry cost'])

            routes['Tcost'] = routes['route cost']+routes['G-entry cost']

            lgk = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, lgk) for i in b]
            lod = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, lod) for i in b]
            
            linestr = pd.DataFrame({'route no':lod,'route step':lgk})
            
            linestr = pd.merge(linestr, Conn.geometry, left_on = 'route step', right_index = True, how = 'left')
            linestr = linestr[['route no','geometry']]
            linestr = gpd.GeoDataFrame(linestr[['route no','geometry']], crs = 4326)
            
            linestr = linestr.dissolve('route no')
            routes2 = pd.merge(routes, linestr, left_on = 'graph_key', right_index = True, how = 'left')
            
            Lines_df = pd.concat([Lines_df, linestr])
            Routes_df = pd.concat([Routes_df, routes2])
            
            print(round(l*num / len(sets)*100,2),'%', 
                  round((time.time() - start_time) / 60,2),'mns')
            time.sleep(time_sleep)
        Routes_df = Routes_df.sort_values('index')
        Routes_df = Routes_df.set_index('index')
        Routes_df = Routes_df.reset_index(drop = True)
        
        Routes_df = Routes_df[Routes_df.columns[~Routes_df.columns.isin(['UGSe_id', 'GrE_id'])]]
        
        print('100 % done',round((time.time() - start_time) / 60,2),'mns')
        
        Routes.append(Routes_df)
        Lines.append(Lines_df)
    return(Routes)

In [9]:
def min_gridUGS_comb (routes, grids, UGS):
    gp_nearest = []
    for i in range(len(routes)):
        gp_nn = routes[i][routes[i]['Tcost'] <= max(thresholds)]
        gp_nn = pd.merge(gp_nn, grids[i]['population'], left_on='Grid_No', right_index = True)
        gp_nn = pd.merge(gp_nn, UGS[i]['park_area'], left_on = 'Park_No', right_index = True)
        gp_nn = gp_nn.reset_index()

        gp_nn = gp_nn.iloc[gp_nn.groupby('gridpark_no')['Tcost'].idxmin()]
        gp_nn.index.name = 'idx'
        gp_nn = gp_nn.sort_values('idx')
        gp_nn = gp_nn.reset_index()
        gp_nearest.append(gp_nn)
    gp_nearest[0].sort_values('Grid_No')
    return(gp_nearest)

In [10]:
def E2SCFA_scores(min_gridUGS_comb, grids, thresholds, cities, 
                  save_path = 'D:/Dumps/GEE-WP Scores/E2SFCA/', grid_size = 100, ext = ''):
    pd.options.display.float_format = '{:20,.2f}'.format
    E2SFCA_cities = []
    E2SFCA_summary = pd.DataFrame()
    for i in range(len(cities)):
        E2SFCA_score = grids[i][['population','geometry']]
        for j in range(len(thresholds)):
            subset = min_gridUGS_comb[i][min_gridUGS_comb[i]['Tcost'] <= thresholds[j]]

            # use gussian distribution: let v= 923325, then the weight for 800m is 0.5
            v = -thresholds[j]**2/np.log(0.5)

            # add a column of weight: apply the decay function on distance
            subset['weight'] = np.exp(-(subset['Tcost']**2/v)).astype(float)
            subset['pop_weight'] = subset['weight'] * subset['population']

            # get the sum of weighted population each green space has to serve.
            s_w_p = pd.DataFrame(subset.groupby('Park_No').sum('pop_weight')['pop_weight'])

            # delete other columns, because they are useless after groupby
            s_w_p = s_w_p.rename({'pop_weight':'pop_weight_sum'},axis = 1)
            middle = pd.merge(subset,s_w_p, how = 'left', on = 'Park_No' )

            # calculate the supply-demand ratio for each green space
            middle['green_supply'] = middle['park_area']/middle['pop_weight_sum']

            # caculate the accessbility score for each green space that each population grid cell could reach
            middle['Sc-access'] = middle['weight'] * middle['green_supply']
            # add the scores for each population grid cell
            pop_score_df = pd.DataFrame(middle.groupby('Grid_No').sum('Sc-access')['Sc-access'])

            # calculate the mean distance of all the green space each population grid cell could reach
            mean_dist = middle.groupby('Grid_No').mean('Tcost')['Tcost']
            pop_score_df['M-dist'] = mean_dist

            # calculate the mean area of all the green space each population grid cell could reach
            mean_area = middle.groupby('Grid_No').mean('park_area')['park_area']
            pop_score_df['M-area'] = mean_area

            # calculate the mean supply_demand ratio of all the green space each population grid cell could reach
            mean_supply = middle.groupby('Grid_No').mean('green_supply')['green_supply']
            pop_score_df['M-supply'] = mean_supply

            pop_score = pop_score_df

            pop_score_df = pop_score_df.join(grids[i]['population'], how = 'right')
            pop_score_df['Sc-norm'] = pop_score_df['Sc-access'] / pop_score_df['population']

            pop_score_df = pop_score_df.loc[:, pop_score_df.columns != 'population']
            pop_score_df = pop_score_df.add_suffix(' '+str(thresholds[j]))
            E2SFCA_score = E2SFCA_score.join(pop_score_df, how = 'left')

            print(thresholds[j], cities[i])

        E2SFCA_score = E2SFCA_score.fillna(0)
        
        if not os.path.exists(save_path+str(grid_size)+'m grids'+'/grid_geoms/'):
            os.makedirs(save_path+str(grid_size)+'m grids'+'/grid_geoms/')
        
        E2SFCA_score.to_file(save_path+str(grid_size)+'m grids'+'/grid_geoms/'+cities[i]+'.gpkg') # Detailed scores
        pop_sum = pd.Series(E2SFCA_score['population'].sum()).astype(int)
        mean_metrics = E2SFCA_score.loc[:, ~E2SFCA_score.columns.isin(['population','geometry'])].mean()
        E2SFCA_sum = pd.concat([pop_sum, mean_metrics])
        E2SFCA_summary = pd.concat([E2SFCA_summary, E2SFCA_sum], axis = 1) # summarized results
        E2SFCA_cities.append(E2SFCA_score)
        
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        E2SFCA_score.loc[:, E2SFCA_score.columns != 'geometry'].to_csv(save_path+cities[i]+'.csv')
    E2SFCA_summary.columns = cities
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    E2SFCA_summary.to_csv(save_path+str(grid_size)+'m grids'+'all_cities'+ext+'.csv')
    E2SFCA_summary
    return({'score summary':E2SFCA_summary,'score detail':E2SFCA_cities})